In [16]:
import pandas as pd
import sqlite3
import os


In [17]:
df = pd.read_csv("retail_sales_dataset.csv")

print("Original Shape:", df.shape)
print("Columns available:\n", df.columns)


Original Shape: (1000, 9)
Columns available:
 Index(['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age',
       'Product Category', 'Quantity', 'Price per Unit', 'Total Amount'],
      dtype='object')


In [18]:
# remove duplicates
df.drop_duplicates(inplace=True)

# handle missing values
df.fillna(method="ffill", inplace=True)

# standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

print("After cleaning:", df.shape)


After cleaning: (1000, 9)


/tmp/ipython-input-1312025370.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [19]:
for col in df.columns:
    if "quantity" in col or "price" in col or "amount" in col:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)


In [20]:
# find quantity and price columns automatically
qty_col = next((c for c in df.columns if "quantity" in c), None)
price_col = next((c for c in df.columns if "price" in c), None)

if qty_col and price_col:
    df["total_sales"] = df[qty_col] * df[price_col]
    df["profit"] = df["total_sales"] * 0.20
    df["segment"] = df["total_sales"].apply(lambda x: "High" if x > 500 else "Low")

df.head()


,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,total_sales,profit,segment
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,30.0,Low
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,200.0,High
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,6.0,Low
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,100.0,Low
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,20.0,Low


In [22]:
df.to_csv("processed_data.csv", index=False)
print("processed_data.csv saved")


processed_data.csv saved


In [23]:
customer_cols = [c for c in df.columns if "customer" in c or "age" in c or "gender" in c]

customers = df[customer_cols].drop_duplicates()

customers.to_csv("customers.csv", index=False)

print("Customers columns:", customer_cols)
customers.head()


Customers columns: ['customer_id', 'gender', 'age']


,customer_id,gender,age
0,CUST001,Male,34
1,CUST002,Female,26
2,CUST003,Male,50
3,CUST004,Male,37
4,CUST005,Male,30


In [24]:
product_cols = [c for c in df.columns if "product" in c or "category" in c or "price" in c]

products = df[product_cols].drop_duplicates()

products.to_csv("products.csv", index=False)

print("Products columns:", product_cols)
products.head()


Products columns: ['product_category', 'price_per_unit']


,product_category,price_per_unit
0,Beauty,50
1,Clothing,500
2,Electronics,30
5,Beauty,30
6,Clothing,25


In [25]:
order_cols = [c for c in df.columns if c not in customer_cols + product_cols]

orders = df[order_cols]

orders.to_csv("orders.csv", index=False)

print("Orders columns:", order_cols)
orders.head()


Orders columns: ['transaction_id', 'date', 'quantity', 'total_amount', 'total_sales', 'profit', 'segment']


,transaction_id,date,quantity,total_amount,total_sales,profit,segment
0,1,2023-11-24,3,150,150,30.0,Low
1,2,2023-02-27,2,1000,1000,200.0,High
2,3,2023-01-13,1,30,30,6.0,Low
3,4,2023-05-21,1,500,500,100.0,Low
4,5,2023-05-06,2,100,100,20.0,Low


In [27]:
conn = sqlite3.connect("database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)

conn.close()

print("database.sqlite created")


database.sqlite created


In [28]:
print("\nFinal Counts:")
print("Customers:", len(customers))
print("Products :", len(products))
print("Orders   :", len(orders))



Final Counts:
Customers: 1000
Products : 15
Orders   : 1000
